In [13]:
from openalea.mtg.mtg import *
from openalea.mtg.algo import ancestors
from openalea.mtg.io import *
from openalea.mtg.traversal import *
from scipy.stats import poisson, binom
import os
import timeit
import sys
from oawidgets.mtg import *
from oawidgets.plantgl import PlantGL
from openalea.mtg.draw import *
import sys

sys.path.append("../../Task_2/src/")

from algo_clustering import *
from algo_distributed_mpi import *
from oawidgets.mtg import plot
from oawidgets.plantgl import PlantGL
from IPython.display import HTML
from IPython.display import IFrame

In [15]:

%%px
from mpi4py import MPI
print(MPI.COMM_WORLD.size)

[stdout:0] 4
[stdout:1] 4
[stdout:2] 4
[stdout:3] 4


In [16]:
from ipyparallel import Client
clients = Client()

In [17]:

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

def f_unc():
    for x in range(10000):
       x+=1

algos = [Best_Fit_Clustering,First_Fit_Clustering, Best_Fit_Clustering_post_order, Best_Fit_Clustering_level_order]
t_size = 99

#nb_cpus = [8,16,32,64,128]
nb_cpus=[4]
nb_tries = 1
for i in range(nb_tries):
    my_mtg = MTG()
    np.random.seed(seed = i)
    dist = poisson(1., loc=1).rvs         
    vid = my_mtg.add_component(my_mtg.root)
    random_tree(my_mtg,vid,nb_children=dist,nb_vertices=t_size)
    for j in range(len(nb_cpus)):
        for algo in algos:
            if rank == 0:
                if my_mtg.property('cluster') != {}:
                    my_mtg.remove_property('cluster')
                if my_mtg.property('sub_tree') != {}:
                    my_mtg.remove_property('sub_tree')
                if my_mtg.property('connection_nodes') != {}:
                    my_mtg.remove_property('connection_nodes')
                if my_mtg.max_scale() - 1 !=  0:
                    my_mtg.remove_scale(my_mtg.max_scale()-1)
                start = MPI.Wtime()
                if algo in algos:
                    if algo != First_Fit_Clustering :
                        algo(my_mtg,nb_cpus[j],0.4)
                    else:
                        algo(my_mtg,nb_cpus[j])
               
                
                else:
                    raise ("Wrong algorithm try one of these ",algos)
                
                end = MPI.Wtime()
                if path.exists('../data/results/' + algo.__name__ + '_partition_time.npy'):
                    with open('../data/results/' + algo.__name__ + '_partition_time.npy','rb') as f:
                        data = np.load(f)

                    data[i,j] = end - start
                    with open('../data/results/' + algo.__name__ + '_partition_time.npy','wb') as f1:
                        np.save(f1,data)      
                else:
                    data = np.zeros([nb_tries,len(nb_cpus)])
                    data[i,j] = end - start
                    with open('../data/results/' + algo.__name__ + '_partition_time.npy','wb') as f1:
                        np.save(f1,data) 
                
                if path.exists('../data/results/' + algo.__name__ + '_longest_path.npy'):
                    with open('../data/results/' + algo.__name__ + '_longest_path.npy','rb') as f:
                        data = np.load(f)
                    #T.insert_scale(T.max_scale(), lambda vid: vid in sub_tree and vid != None)

                    data[i,j] = longest_path(my_mtg,nb_cpus[j])
                    with open('../data/results/' + algo.__name__ + '_longest_path.npy','wb') as f1:
                        np.save(f1,data)      
                else:
                    data = np.zeros([nb_tries,len(nb_cpus)])
                    data[i,j] = longest_path(my_mtg,nb_cpus[j])
                    with open('../data/results/' + algo.__name__ + '_longest_path.npy','wb') as f1:
                        np.save(f1,data) 

                

        
                print("Partitoning finished for ",algo.__name__," is", end-start)
                print("---------------------------------------------------------------")
            comm.Barrier()
           
                
            distributed_tree_traversal_bottom_up(my_mtg,algo,j,f_unc,i,nb_tries)   
            distributed_tree_traversal_top_down(my_mtg,algo,j,f_unc,i,nb_tries) 
         





Partitoning finished for  Best_Fit_Clustering  is 0.0015099048614501953
---------------------------------------------------------------


Exception: Invalid rank, error stack:
PMPI_Irecv(156): MPI_Irecv(buf=0x5647ae99a590, count=32768, MPI_BYTE, src=3, tag=34, MPI_COMM_WORLD, request=0x7effa06072e0) failed
PMPI_Irecv(99).: Invalid rank has value 3 but must be nonnegative and less than 1

In [ ]:
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_Paper,"top_down")
end_time = timeit.default_timer()
print("Time for top_down: ",end_time - start_time," seconds")
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_Paper,"bottom_up")
end_time = timeit.default_timer()
print("Time for bottom up: ",end_time - start_time," seconds")
IFrame(src='../data/plots/Best_Fit_Clustering_Paper_dependecy.html', width=1000, height=500)

In [ ]:
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_Queue,"Top_down")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_Queue,"bottom_up")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")
IFrame(src='../data/plots/Best_Fit_Clustering_Queue_dependecy.html', width=1000, height=500)

In [ ]:
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,First_Fit_Clustering_Paper,"top_down")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,First_Fit_Clustering_Paper,"bottom_up")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")
IFrame(src='../data/plots/First_Fit_Clustering_Paper_dependecy.html', width=1000, height=500)

In [ ]:
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_Queue_1,"top_down")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_Queue_1,"bottom_up")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")
IFrame(src='../data/plots/Best_Fit_Clustering_Queue_1_dependecy.html', width=1000, height=500)

In [ ]:
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_level_order,"top_down")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")
start_time = timeit.default_timer()
distributed_tree_traversal(test_mtg,Best_Fit_Clustering_level_order,"bottom_up")
end_time = timeit.default_timer()
print("Time: ",end_time - start_time," seconds")

In [ ]:

IFrame(src='../data/plots/Best_Fit_Clustering_level_order_dependecy.html', width=1000, height=500)